# Tested parameter space in Moore et al. (2024; submitted to ApJ):
## Host star = M8 (stellar tracks from Baraffe et al. 2015)
## Water inventory = {2, 4, 6, 8} Earth Oceans, but here we use water mass fraction (scaled from 2, 4, 6, 8 Earth Oceans for 1 Earth Mass)
## Orbital distance = {Hot Inner, Mid, Cold Outer} HZ edge (@t=4.5 Gyr, from Kopparapu et al. 2013)
# ---------------------------------

# This notebook determine T_surfs based on OLR (=incoming flux): when below Simpson-Nakajima limit (325 W/m^2), T_surf = 293.15 K; during runaway greenhouse of T_surf = 1800 K (same as Barth et al. 2020).

# ASSUMPTIONS:
## No regassing during runaway greenhouse; all water in atmosphere (i.e., no surface water), and regassing parameterization relies on subduction. 
## Minimum water in mantle set to "desiccation limit" of Moore & Cowan (2020); lower limit prevents mantle viscosity --> infinity in calculations. 

## MO solidification timescale is the same as calculcated RG timescale.
## Use a CONSTANT thermospheric temperature = 400 K to calculate diffusion-limited escape, as in Luger & Barnes (2015)
## - Pure H2O atmosphere; we only track the mass of H2O in each reservoir throughout the model

######################################################################################################################

# Initialize notebook; import required functions/packages, and read in input parameters/variables from associated .txt files. The majority of these parameters are outlined in Table C1 of Moore & Cowan (2020). 

In [ ]:
%pylab inline
import math
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import gridspec
from pylab import *
from scipy.integrate import ode
import time
import os.path
import importlib

Import all the constants and functions from local modules.

In [ ]:
from MOparams import *
from MOfunc import *

# Reload the local modules if anything has been changed
%load_ext autoreload
%autoreload 2

## Please modify the following settings before running the simualtion.

In [ ]:
# Path to save following simulations to file. -- REPLACE WITH YOUR OWN DIRECTORY IF SAVING

savepath = '/Users/admin/Desktop/FOR_GITHUB/'
savepath.replace("\t","\\t")
savepath.replace("\b","\\b")
savepath.replace("\r","\\r")
savepath.replace("\n","\\n")

In [ ]:
# Do you have MOresults and MOcyclingresults data files in your local directory and want to skip the MO phase in the simulation?
# Skip the MO simulation can save runtime, but errors may occur if your local data file is generated with parameters \
# inconsistent with this notebook.

# True == Skip the MO simulation

have_MO_data = False

# Run MO simulation here.
## Initialize all variables for the MO simulations.

In [ ]:
# Stellar evolution and surface temperature calculations

# Stellar evolution track, from Baraffe et al. 2015 (BHAC15)
# Read in stellar evolution track data
hostfile = 'BHAC15_0.09Msun.txt' # corresponds to M8 host star

# Parameters: Stellar type, Orbital distance, total initial water, MO/BMO solidification timescale.

## Set the initial amount of water in the MO, and calculate the initial concentration.

In [ ]:
# Initial mass of water in MO, and initial concentration in the MO.
M_init_array = NUM_EARTH_OCEANS*M #np.array([2., 4., 6., 8.])*1.4e21 #[kg]; 1.4e21 kg = 1 Earth Ocean
C_0_array = (3.*M_init_array)/(4.*np.pi*rho*(R_p**3. - R_c**3.))
print(M_init_array/1.4e21, 'Earth Oceans')

## Parameterization of water in each inventory during MO solidification.


## Functions required for atmospheric loss calculations.

In [ ]:
# Orbital distances; used to determine Inner HZ, Mid HZ, and Outer HZ orbital positions.
# Read in Kopparapu et al. (2013) HZ data, for different stellar hosts, implemented in the f_a_orb() function.

HZ_data_file = "HZs_orbits.dat"

In [ ]:
print(HABITABLE_ZONE)

In [ ]:
# Parameter settings please consult MOfunc.py, starting line 486
star_age, Lbol, Lbol_Ls_star, T_eff_star = hoststar(hostfile)
T_eff_star_45 = np.interp(4.5e9*year, star_age, T_eff_star)
Lbol_Ls_star_45 = np.interp(4.5e9*year, star_age, Lbol_Ls_star)
# Roughly find limits at t = 4.5 Gyr (~age of Earth)

a_RV, a_RG, a_MG, a_EM = f_a_orb(4.5e9*year, HZ_data_file, hostfile)

if HABITABLE_ZONE == 'ALL':
    
    a_orb_array = np.array([a_RG, (a_RG+a_MG)/2., a_MG])
    a_orb_labels = ['Inner', 'Middle', 'Outer']
    
else:

    # Take runaway greenhouse and maximum greenhouse limits as Inner/Outer HZ, and then the middle between them.
    possible_habitable_zones = {"Inner" : a_RG, "Middle" : (a_RG+a_MG)/2., "Outer" : a_MG}
    a_orb_array = np.array([val for (key, val) in possible_habitable_zones.items() if key == HABITABLE_ZONE])

    habitable_zone_labels = {"Inner" : "Runaway_GH", "Middle" : "Middle", "Outer" : "Maximum_GH"}
    a_orb_labels = np.array([val for (key, val) in habitable_zone_labels.items() if key == HABITABLE_ZONE])

In [ ]:
print(a_orb_array, 'AU')
print(a_orb_labels, 'HZ')

In [ ]:
#M_init_array is the TOTAL initial water inventory, which starts in the MO; M_init_file should match that array.
M_init_file = NUM_EARTH_OCEANS_FILE #np.array([2., 4., 6., 8.]) #[Earth Oceans]
print(M_init_file, 'Earth Oceans')
dt_MO = 2.03e3*year

RG_t_array = np.arange(0, 1.0e9*year, dt_MO)
tau_RG_array = np.zeros(len(a_orb_array))

# calculating the time at which the runaway greenhouse and magma ocean end, based on the incoming flux for each orbital distance in a_orb_array. 
for i in range(0, len(a_orb_array)):
    RG_TOA_flux_array = S_0(RG_t_array, a_orb_array[i]*1.496e11)
    for j in range(0, len(RG_t_array)):
        if RG_TOA_flux_array[j]/4. < 325./(1-alb):
            tau_RG_array[i] = RG_t_array[j]
            exit_idx = i
            break

tau_MO_array = tau_RG_array
tau_MO_file = tau_MO_array/1.0e6/year

## Begin loop for the water inventories over time during the MO.

In [ ]:
# This cell generates the MO results, i.e. the sans BMO model.
# Please ignore this cell if you have MOresults and MOcycling results at your local directory.
if have_MO_data != True:
    dt_MO = 2.0e3*year #2000 yr per timestep
    
    
    # Arrays to save final water inventories at end of MO simulation.
    W_m_init_array = np.zeros((len(M_init_array), len(a_orb_array)))
    W_s_init_array = np.zeros((len(M_init_array), len(a_orb_array)))

    # Path to save MO simulations to file. 
    save_path = savepath

    # Loop over water inventory array. 
    for kdx in range(0, len(M_init_array)): 
        # Loop over orbital distance array. 
        for ldx in range(0,len(a_orb_array)):
                
            t_MO_array = np.arange(0., (tau_MO_array[ldx]+dt_MO), dt_MO) # equivalent to MO solidification timescale
            r_array = R_p - d_MO(t_MO_array, tau_MO_array[ldx], M)

            # Can comment out if only running a single simulation, but used to save results to file.
            save_file = f'MOresults_{int(M/M_E)}_earth_mass_D_of_{D}_' + str(np.int(NUM_EARTH_OCEANS_FILE[kdx])) + '_oceans_' + a_orb_labels[ldx] + '_hz.txt'
            filename = os.path.join(save_path, save_file)
            f = open(filename, 'w')
            
            # 4 TOTAL reservoirs, for mass balance throughout
            M_MO_array = np.zeros(len(r_array)) #magma ocean
            M_SM_array = np.zeros(len(r_array)) #solid mantle
            M_atmo_array = np.zeros(len(r_array)) #atmosphere
            M_lost_array = np.zeros(len(r_array)) #space "sink" for lost water

            # Other useful arrays for saving/plotting later
            EL_loss_MO_array = np.zeros(len(r_array)) #EL loss rate
            DL_loss_MO_array = np.zeros(len(r_array)) #DL loss rate
            loss_MO_array = np.zeros(len(r_array)) #actual loss rate
            TOA_flux_MO_array = np.zeros(len(r_array)) #top-of-atmosphere flux during MO
            a_orb = a_orb_array[ldx]*1.496e11 #[m]
            
            # Track MO temperature as a function of r/t.
            T_MO_array = np.zeros(len(t_MO_array))

            # Keep track of the total water inventory, for mass balance purposes throughout.
            M_total = M_init_array[kdx] #[kg]

            # Initial concentration; needs to be altered from array if MO begins simulation saturated.
            C_initial = C_0_array[kdx]

            # First, need to check if MO holds MORE water than saturation limit; if it does, degas the excess into
            # atmosphere FIRST before running simulations.
            if C_initial > C_sat:

                MO_excess = (C_initial-C_sat)*(4.*np.pi*rho*(R_p**3. - R_c**3.))/3.

                M_MO_array[0] = M_total - MO_excess
                M_SM_array[0] = 0.
                M_atmo_array[0] = MO_excess

                C_initial = (3.*M_MO_array[0])/(4.*np.pi*rho*(R_p**3. - R_c**3.))

            else:

                # Set initial parameters; all water in MO:
                M_MO_array[0] = M_total
                M_SM_array[0] = 0.
                M_atmo_array[0] = 0.
            
            EL_loss_MO_array[0] = loss_rate_EL_MO(F_XUV(t_MO_array[0], a_orb), M)
            DL_loss_MO_array[0] = loss_rate_DL_MO(M)
            loss_MO_array[0] = f_loss_MO(t_MO_array[0], M, a_orb)
            TOA_flux_MO_array[0] = S_0(t_MO_array[0], a_orb)
            T_MO_array[0] = T_MO(t_MO_array[0], tau_MO_array[ldx], M)
            
            # Comment in if writing to file; initial values for all above defined arrays.
            f.write(str(t_MO_array[0]) + '\t' + str(r_array[0]) + '\t' + str(T_MO_array[0]) + '\t' + str(M_MO_array[0]) +'\t' + \
                    str(M_SM_array[0]) + '\t' + str(M_atmo_array[0]) + '\t' + str(EL_loss_MO_array[0]) + '\t' + \
                    str(DL_loss_MO_array[0]) + '\t' +str(loss_MO_array[0]) + '\t' + str(TOA_flux_MO_array[0]) + '\t' +\
                    str(M_lost_array[0]) + '\n')
            
            
            # Begin loop, advancing r(t) forward in time by timestep dt_MO.
            for idx in range(1,len(r_array)):

                EL_loss_MO_array[idx] = loss_rate_EL_MO(F_XUV(t_MO_array[idx], a_orb), M)
                DL_loss_MO_array[idx] = loss_rate_DL_MO(M)
                loss_MO_array[idx] = f_loss_MO(t_MO_array[idx], M, a_orb)
                TOA_flux_MO_array[idx] = S_0(t_MO_array[idx], a_orb)
                T_MO_array[idx] = T_MO(t_MO_array[idx], tau_MO_array[ldx], M)
                
                #MO undersaturated with water, i.e., r <= R_sat
                if r_array[idx] <= R_sat(C_initial, M):

                    M_MO_array[idx] = M_MO_unsat(C_initial, r_array[idx], M)

                    M_SM_array[idx] = M_SM_unsat(C_initial, r_array[idx], M)

                    M_atmo_array[idx] = 0.

                # MO saturated/supersaturated with water (r > R_sat); atmosphere forms    
                else: #r_array[idx] > R_sat()

                    M_MO_array[idx] = M_MO_sat(r_array[idx], M)

                    M_SM_array[idx] = M_SM_sat(C_initial, r_array[idx], M)

                    M_atmo_array[idx] = M_total - M_SM_sat(C_initial, r_array[idx], M) - M_MO_sat(r_array[idx], M) - M_lost_array[idx-1]

                   # Now add loss:
                    if M_atmo_array[idx] <= (loss_MO_array[idx]*dt_MO):

                        M_lost_array[idx] = M_lost_array[idx-1] + M_atmo_array[idx]
                        M_atmo_array[idx] = 0.  #no water in atmosphere; all lost to space

                    else: #M_atmo_array[idx] > (loss_MO_array[idx]*dt_MO)

                        M_atmo_array[idx] = M_atmo_array[idx] - (loss_MO_array[idx]*dt_MO)
                        M_lost_array[idx] = M_lost_array[idx-1] + (loss_MO_array[idx]*dt_MO)

                # Comment the below line back in, to write results to file, for each timestep in loop.
                f.write(str(t_MO_array[idx]) + '\t' + str(r_array[idx]) + '\t' + str(T_MO_array[idx]) + '\t' + str(M_MO_array[idx]) +'\t' + \
                    str(M_SM_array[idx]) + '\t' + str(M_atmo_array[idx]) + '\t' + str(EL_loss_MO_array[idx]) + '\t' + \
                    str(DL_loss_MO_array[idx]) + '\t' +str(loss_MO_array[idx]) + '\t' + str(TOA_flux_MO_array[idx]) + '\t' +\
                    str(M_lost_array[0]) + '\n')
                
                
             # Save final values to initial arrays (at end of MO/beginning of cycling) for mantle and surface reservoirs.
            W_m_init_array[kdx, ldx] = M_SM_array[-1]
            W_s_init_array[kdx, ldx] = M_atmo_array[-1]

            # Comment back in if writing to file.
            f.close()

# Now we can define cycling-specific equations, to be used in the simulations for the rest of the M-Earths lifetime.
## Same parameter space over tau_MO, water inventory, and orbital distances is followed.
## Recall that the final water inventories in solid mantle and surface will be used as input parameters for cycling simulations.

# Cycling Functions; see Moore & Cowan (2020) for thermal evolution, and Moore, Cowan & Boukare (2023) for details of cycling. 

In [ ]:
## Orbital distances; repeated here in case prior MO simulation is not followed.
# Read in Kopparapu+(2013) HZ data, for different stellar hosts.

data = np.loadtxt('HZs_orbits.dat', skiprows=2)
T_eff_kopp = data[:,0] #[K]
# All below values give S_eff -- see Eqn.(2) of Kopparapu+(2013)
RV_kopp = data[:,1] #Recent Venus
RG_kopp = data[:,2] #Runaway Greenhouse
MG_kopp = data[:,3] #Maximum Greenhouse
EM_kopp = data[:,4] #Early Mars

# Time-dependent Cycling & Loss Simulations

## Change the cell below from MARKDOWN to CODE if running simulations.
## If just reading in files (below this simulation loop), run all cells above this point, and just read in MO & cycling data from files.
## ***NOTE FOR THIS CELL: YOU MAY GET SOME ERRORS WHILE RUNNING THIS INTEGRATION; THE LOOP CONTINUES IN A DIFFERENT MANOR WHEN IF THE VODE FAILS. EVEN WITH ERRORS, LET IT RUN, AND DOUBLE-CHECK THE RESULTS WHEN PLOTTED TO FIGURE AFTERWARDS -- IF IT IS SMOOTH THEN NO PROBLEM! OTHERWISE, NEED TO PLAY WITH THE INTEGRATION IN THE BELOW CELL

In [ ]:
# MO phase cycling simulation can be skipped if local data files are present.
if have_MO_data != True:
    
    for index in range(1):
        # Loop over initial water inventories.
        for kdx in range(0, len(M_init_array)):

            # Loop over orbital distances.
            for ldx in range(0, len(a_orb_array)):

                # Set initial conditions.
                t0 = tau_MO_array[ldx] #START AT END OF MO PHASE, is not set equal to tau_RG
                # z0 = [mantle temperature, mantle water, surface water]
                z0 = [3000., W_m_init_array[kdx, ldx], W_s_init_array[kdx, ldx]] #[K] [kg] [kg]

                # Atmosphere initial conditions.
                m = m_H2O #molecular mass of water 
                a_orb = a_orb_array[ldx]*1.496e11 #[m]
                alb = 0.3 #albedo; redefined here for clarity
                T_surf_0 = T_surf_OLR(t0, alb, a_orb)
                T_strat_0 = T_strat(t0, a_orb)
                # Define max time, timestep, arrays to be filled within the integration loop.
                t1 = 5.0e9*year #5 Gyr
                dt = 2.0e4*year #20,000 yr per timestep

                # Set up function to be integrated.
                r = ode(f_cycling_MO).set_integrator('vode')
                r.set_initial_value(z0, t0).set_f_params(T_surf_0,T_strat_0,alb,dt,M,a_orb)

                # Define arrays to be filled within the integration loop.
                t_array = np.zeros(int((t1-t0)/dt)+1)
                # Cycling
                T_array = np.zeros(int((t1-t0)/dt)+1)
                W_m_array = np.zeros(int((t1-t0)/dt)+1)
                W_s_array = np.zeros(int((t1-t0)/dt)+1)
                regas_array = np.zeros(int((t1-t0)/dt)+1)
                degas_array = np.zeros(int((t1-t0)/dt)+1)
                # Atmosphere
                T_surf_array = np.zeros(int((t1-t0)/dt)+1)
                T_strat_array = np.zeros(int((t1-t0)/dt)+1)
                TOA_flux_array = np.zeros(int((t1-t0)/dt)+1)
                loss_array = np.zeros(int((t1-t0)/dt)+1)
                EL_array = np.zeros(int((t1-t0)/dt)+1)
                DL_array = np.zeros(int((t1-t0)/dt)+1)

                # Write results to file, for plotting later. -- CHANGE PATH IF SAVING TO FILE. 
                save_path = savepath
                save_file = f'cyclingresults_{int(M/M_E)}_earth_mass_D_of_{D}_' + str(np.int(NUM_EARTH_OCEANS_FILE[kdx])) + '_oceans_' + a_orb_labels[ldx] + '_hz.txt'
                filename = os.path.join(save_path, save_file)
                f = open(filename, 'w')

                # Initial values in the arrays.
                t_array[0] = t0
                # Cycling
                T_array[0] = z0[0]
                W_m_array[0] = z0[1]
                W_s_array[0] = z0[2]
                regas_array[0] = f_regas(t0, z0[0], z0[1], z0[2], T_surf_0, alb, M, a_orb)
                degas_array[0] = f_degas(t0, z0[0], z0[1], z0[2], T_surf_0, M)

                # Atmosphere
                T_surf_array[0] = T_surf_0
                T_strat_array[0] = T_strat_0
                TOA_flux_array[0] = S_0(t0, a_orb)
                loss_array[0] = f_loss(t0, z0[1], z0[2], alb, dt, M, a_orb)
                EL_array[0] = loss_rate_EL(F_XUV(t0, a_orb), Rp(M), M) #[kg/s]
                DL_array[0] = loss_rate_DL(M, 400) #[kg/s], assumes thermospheric temperature at 400K for all time.

                # Write initial values to file. Comment back in if saving to file.
                f.write(str(t_array[0]) + '\t' + str(T_array[0]) + '\t' + str(W_m_array[0]) +'\t' + str(W_s_array[0]) + \
                        '\t' + str(degas_array[0]) + '\t' + str(regas_array[0]) + '\t' + str(T_surf_array[0]) + '\t' + \
                        str(T_strat_array[0]) + '\t' + str(TOA_flux_array[0]) + '\t' + str(loss_array[0]) + '\t' + \
                        str(EL_array[0]) + '\t' + str(DL_array[0]) + '\n')

                # Integrate the above function.
                idx = 1
                start_time = time.time() #time how long each loop takes
                for idx in range(1,len(t_array)):

                    if r.successful() == True:
                        r.integrate(r.t+dt)
                        t_array[idx] = r.t
                        T_array[idx] = r.y[0]
                        W_m_array[idx] = r.y[1]
                        W_s_array[idx] = r.y[2]

                        T_surf_array[idx] = T_surf_OLR(r.t, alb, a_orb)
                        T_strat_array[idx] = T_strat(r.t, a_orb)
                        TOA_flux_array[idx] = S_0(r.t, a_orb)

                        regas_array[idx] = f_regas(r.t, r.y[0], r.y[1], r.y[2], T_surf_array[idx], alb, M, a_orb)
                        degas_array[idx] = f_degas(r.t, r.y[0], r.y[1], r.y[2], T_surf_array[idx], M)
                        loss_array[idx] = f_loss(r.t, r.y[1], r.y[2], alb, dt, M, a_orb)
                        EL_array[idx] = loss_rate_EL(F_XUV(r.t, a_orb), Rp(M), M) #[kg/s]
                        DL_array[idx] = loss_rate_DL(M, T = 400) #[kg/s]

                        # Write current values to file. Comment back in if writing to file.
                        f.write(str(t_array[idx]) + '\t' + str(T_array[idx]) + '\t' + str(W_m_array[idx]) +'\t' + str(W_s_array[idx]) + \
                            '\t' + str(degas_array[idx]) + '\t' + str(regas_array[idx]) + '\t' + str(T_surf_array[idx]) + '\t' + \
                            str(T_strat_array[idx]) + '\t' + str(TOA_flux_array[idx]) + '\t' + str(loss_array[idx]) + '\t' + \
                            str(EL_array[idx]) + '\t' + str(DL_array[idx]) + '\n')

                    elif r.successful() == False: #slightly less elegant, more direct way

                        t_array[idx] = t_array[idx-1] + dt
                        T_array[idx] = T_array[idx-1] + \
                            f_delta_temp(t_array[idx-1], T_array[idx-1], W_m_array[idx-1], W_s_array[idx-1], T_surf_array[idx-1], M)*dt
                        # Basal magma ocean variables required for calculation of W_m_array.
                        W_m_array[idx] = W_m_array[idx-1] + \
                            f_delta_W_m_MO(t_array[idx-1], T_array[idx-1], W_m_array[idx-1], W_s_array[idx-1], T_surf_array[idx-1], alb, dt, M, a_orb)*dt
                        W_s_array[idx] = W_s_array[idx-1] + \
                            f_delta_W_s_MO(t_array[idx-1], T_array[idx-1], W_m_array[idx-1], W_s_array[idx-1], T_surf_array[idx-1], T_strat_array[idx-1], alb, dt, M, a_orb)*dt

                        T_surf_array[idx] = T_surf_OLR(t_array[idx], alb, a_orb)
                        T_strat_array[idx] = T_strat(t_array[idx], a_orb)
                        TOA_flux_array[idx] = S_0(t_array[idx], a_orb)

                        regas_array[idx] = f_regas(t_array[idx], T_array[idx-1], W_m_array[idx], W_s_array[idx], T_surf_array[idx], alb, M, a_orb)
                        degas_array[idx] = f_degas(t_array[idx], T_array[idx-1], W_m_array[idx], W_s_array[idx], T_surf_array[idx], M)
                        loss_array[idx] = f_loss(t_array[idx], W_m_array[idx], W_s_array[idx], alb, dt, M, a_orb)
                        EL_array[idx] = loss_rate_EL(F_XUV(t_array[idx], a_orb), Rp(M), M) #[kg/s]
                        DL_array[idx] = loss_rate_DL(M, T = 400) #[kg/s]

                        # Write current values to file. Comment back in if writing to file.
                        f.write(str(t_array[idx]) + '\t' + str(T_array[idx]) + '\t' + str(W_m_array[idx]) +'\t' + str(W_s_array[idx]) + \
                            '\t' + str(degas_array[idx]) + '\t' + str(regas_array[idx]) + '\t' + str(T_surf_array[idx]) + '\t' + \
                            str(T_strat_array[idx]) + '\t' + str(TOA_flux_array[idx]) + '\t' + str(loss_array[idx]) + '\t' + \
                            str(EL_array[idx]) + '\t' + str(DL_array[idx]) + '\n')

                if W_m_array[-1]+W_s_array[-1] <= 1.29e16: #desiccation limit from Moore & Cowan (2020); can also be checked later if data saved to file
                    print('Planet Desiccated')

                # Comment back in if saving to file.
                f.close()

                end_time = time.time()
                print(end_time-start_time, ' s') #print the amount of time it took for the loop to run.